In [10]:
using Compat
using MomentumBasedControl
using RigidBodyDynamics
using RigidBodyDynamics.Contact
using AtlasRobot
using TypeSortedCollections
using MathOptInterface

In [11]:
mechanism = AtlasRobot.mechanism()
remove_fixed_tree_joints!(mechanism);

In [12]:
# add environment
rootframe = root_frame(mechanism)
ground = HalfSpace3D(Point3D(rootframe, 0., 0., 0.), FreeVector3D(rootframe, 0., 0., 1.))
add_environment_primitive!(mechanism, ground);

In [20]:
const SparseSymmetric64 = Symmetric{Float64,SparseMatrixCSC{Float64,Int}}

struct StandingController{N, L<:MomentumBasedController}
    foottasks::Dict{Symbol, SpatialAccelerationTask}
    pelvistask::SpatialAccelerationTask
    jointtasks::Dict{JointAccelerationTask, SparseSymmetric64}
    contactsettings::Dict{RigidBody{Float64}, Vector{ContactSettings{N}}}
    lowlevel::L
end

function StandingController{N}(
            mech::Mechanism,
            optimizer::MathOptInterface.AbstractOptimizer,
            feet::Dict{Symbol, RigidBodyDynamics.RigidBody{Float64}},
            pelvis::RigidBody{Float64},
            contactinfo::Dict{RigidBody{Float64}, Vector{ContactInfo}}) where N
    world = root_body(mech)
    revolutejoints = filter(j -> joint_type(j) isa Revolute, tree_joints(mech))
    foottasks = Dict(side => SpatialAccelerationTask(mech, path(mech, world, foot)) for (side, foot) in feet)
    pelvistask = SpatialAccelerationTask(mech, path(mech, world, pelvis), linearrows=1:0)
    jointtasks = Dict(JointAccelerationTask(mech, j) => Symmetric(speye(1)) for j in revolutejoints)
    contactsettings = Dict(b => ContactSettings{N}.(infos) for (b, infos) in contactinfo)
    contactsettingsvec = vcat(values(contactsettings)...)
    motionconstraints = AbstractMotionTask[]
    append!(motionconstraints, values(foottasks))
    push!(motionconstraints, pelvistask)
    lowlevel = MomentumBasedController(mechanism, optimizer, contactsettingsvec, motionconstraints, jointtasks)
    controller = StandingController{N, typeof(lowlevel)}(foottasks, pelvistask, jointtasks, contactsettings, lowlevel)
    set_joint_accel_regularization!(controller, 0.0)
    controller
end

In [21]:
function set_joint_accel_regularization!(controller::StandingController, weight::Number)
    for taskweight in values(controller.jointtasks)
        taskweight.data[:] = weight
    end
    nothing
end

set_joint_accel_regularization! (generic function with 1 method)

In [22]:
feet = Dict(:left => findbody(mechanism, "l_foot"), :right => findbody(mechanism, "r_foot"))
pelvis = findbody(mechanism, "pelvis")
contactinfo = Dict{RigidBody{Float64}, Vector{ContactInfo}}()
μ = 0.8
for body in bodies(mechanism)
    points = RigidBodyDynamics.location.(RigidBodyDynamics.contact_points(body))
    if !isempty(points)
        normal = FreeVector3D(default_frame(body), 0.0, 0.0, 1.0)
        contactinfo[body] = [ContactInfo(point, normal, μ) for point in points]
    end
end
using OSQP.MathOptInterfaceOSQP
optimizer = OSQPOptimizer();

In [34]:
controller = StandingController{4}(mechanism, optimizer, feet, pelvis, contactinfo);